# Predicting Crime in US Communites
## Team member: Bibata Rabba Idi, Fatima Javid, JianHui (Jake) Li 

In [167]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import scipy.stats as stats
from sklearn.model_selection import GridSearchCV
import pickle

%matplotlib inline

In [168]:
# Load data
df = pd.read_csv('crimedata.csv')
df.head()

,communityName,state,countyCode,communityCode,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,...,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop
0,BerkeleyHeightstownship,NJ,39.0,5320.0,11980,3.10,1.37,91.78,6.50,1.88,...,14.0,114.85,138.0,1132.08,16.0,131.26,2.0,16.41,41.02,1394.59
1,Marpletownship,PA,45.0,47616.0,23123,2.82,0.80,95.57,3.44,0.85,...,57.0,242.37,376.0,1598.78,26.0,110.55,1.0,4.25,127.56,1955.95
2,Tigardcity,OR,NaN,NaN,29344,2.43,0.74,94.33,3.43,2.35,...,274.0,758.14,1797.0,4972.19,136.0,376.30,22.0,60.87,218.59,6167.51
3,Gloversvillecity,NY,35.0,29443.0,16656,2.40,1.70,97.35,0.50,0.70,...,225.0,1301.78,716.0,4142.56,47.0,271.93,NaN,NaN,306.64,NaN
4,Bemidjicity,MN,7.0,5068.0,11245,2.76,0.53,89.16,1.17,0.52,...,91.0,728.93,1060.0,8490.87,91.0,728.93,5.0,40.05,NaN,9988.79


In [169]:
fig = px.histogram(df, y='state', height=900).update_yaxes(categoryorder='total ascending')

fig.show()

In [170]:
# Shape of dataframe
df.shape

(2215, 146)

In [171]:
# Number of nulls from each column
print(df.isnull().sum())

# Total number of nulls
print('\nTotal number of nulls:', df.isnull().sum().sum())

communityName             0
state                     0
countyCode             1221
communityCode          1224
population                0
                       ... 
autoTheftPerPop           3
arsons                   91
arsonsPerPop             91
ViolentCrimesPerPop     221
nonViolPerPop            97
Length: 146, dtype: int64

Total number of nulls: 44592


In [172]:
# Make a list of columns that contain nulls
cols_with_nulls = df.columns[df.isna().any()].tolist()
print(cols_with_nulls, '\n')

# Show number of nulls for each column that contain nulls
print(df[cols_with_nulls].isnull().sum())

['countyCode', 'communityCode', 'OtherPerCap', 'LemasSwornFT', 'LemasSwFTPerPop', 'LemasSwFTFieldOps', 'LemasSwFTFieldPerPop', 'LemasTotalReq', 'LemasTotReqPerPop', 'PolicReqPerOffic', 'PolicPerPop', 'RacialMatchCommPol', 'PctPolicWhite', 'PctPolicBlack', 'PctPolicHisp', 'PctPolicAsian', 'PctPolicMinor', 'OfficAssgnDrugUnits', 'NumKindsDrugsSeiz', 'PolicAveOTWorked', 'PolicCars', 'PolicOperBudg', 'LemasPctPolicOnPatr', 'LemasGangUnitDeploy', 'PolicBudgPerPop', 'rapes', 'rapesPerPop', 'robberies', 'robbbPerPop', 'assaults', 'assaultPerPop', 'burglaries', 'burglPerPop', 'larcenies', 'larcPerPop', 'autoTheft', 'autoTheftPerPop', 'arsons', 'arsonsPerPop', 'ViolentCrimesPerPop', 'nonViolPerPop'] 

countyCode              1221
communityCode           1224
OtherPerCap                1
LemasSwornFT            1872
LemasSwFTPerPop         1872
LemasSwFTFieldOps       1872
LemasSwFTFieldPerPop    1872
LemasTotalReq           1872
LemasTotReqPerPop       1872
PolicReqPerOffic        1872
PolicPer

In [173]:
# Remove crime columns below PolicBudgPerPop and OtherPerCap from the list
del cols_with_nulls[25:]
del cols_with_nulls[2]

# Drop columns using cols_with_nulls
print('Columns before dropping:', df.shape[1])
df.drop(cols_with_nulls, axis=1, inplace=True)
print('Columns after dropping:', df.shape[1])

Columns before dropping: 146
Columns after dropping: 122


In [174]:
# Fill in Na values with -999
cols_with_nulls_2 = df.columns[df.isna().any()].tolist()
df[cols_with_nulls_2] = df[cols_with_nulls_2].fillna(-999)

In [175]:
# Check number of nulls
df.isnull().sum().sum()

0

In [176]:
# Check for duplicates
df.duplicated().sum()

0

In [177]:
# Create a column for the sum of all crimes
df['total_crime'] = df['murders'] + df['rapes'] + df['robberies'] + df['assaults'] + df['burglaries'] + df['larcenies'] + df['autoTheft'] + df['arsons']

In [178]:
df.head()

,communityName,state,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,...,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop,total_crime
0,BerkeleyHeightstownship,NJ,11980,3.10,1.37,91.78,6.50,1.88,12.47,21.44,...,114.85,138.0,1132.08,16.0,131.26,2.0,16.41,41.02,1394.59,175.0
1,Marpletownship,PA,23123,2.82,0.80,95.57,3.44,0.85,11.01,21.30,...,242.37,376.0,1598.78,26.0,110.55,1.0,4.25,127.56,1955.95,490.0
2,Tigardcity,OR,29344,2.43,0.74,94.33,3.43,2.35,11.36,25.88,...,758.14,1797.0,4972.19,136.0,376.30,22.0,60.87,218.59,6167.51,2308.0
3,Gloversvillecity,NY,16656,2.40,1.70,97.35,0.50,0.70,12.55,25.20,...,1301.78,716.0,4142.56,47.0,271.93,-999.0,-999.00,306.64,-999.00,42.0
4,Bemidjicity,MN,11245,2.76,0.53,89.16,1.17,0.52,24.46,40.53,...,728.93,1060.0,8490.87,91.0,728.93,5.0,40.05,-999.00,9988.79,266.0


In [179]:
# Check total crime distribution using histogram
fig = px.histogram(df, x='total_crime', nbins=20, template='presentation')

fig.show()

In [180]:
fig = px.box(df, y='total_crime', template='presentation')

fig.show()

In [181]:
# Keep only records where total_crime is greater than 0
df = df[df['total_crime'] > 0]

In [182]:
# Define all_features using all the columns in the dataframe
all_features = df.columns.tolist()
print(all_features)

['communityName', 'state', 'population', 'householdsize', 'racepctblack', 'racePctWhite', 'racePctAsian', 'racePctHisp', 'agePct12t21', 'agePct12t29', 'agePct16t24', 'agePct65up', 'numbUrban', 'pctUrban', 'medIncome', 'pctWWage', 'pctWFarmSelf', 'pctWInvInc', 'pctWSocSec', 'pctWPubAsst', 'pctWRetire', 'medFamInc', 'perCapInc', 'whitePerCap', 'blackPerCap', 'indianPerCap', 'AsianPerCap', 'OtherPerCap', 'HispPerCap', 'NumUnderPov', 'PctPopUnderPov', 'PctLess9thGrade', 'PctNotHSGrad', 'PctBSorMore', 'PctUnemployed', 'PctEmploy', 'PctEmplManu', 'PctEmplProfServ', 'PctOccupManu', 'PctOccupMgmtProf', 'MalePctDivorce', 'MalePctNevMarr', 'FemalePctDiv', 'TotalPctDiv', 'PersPerFam', 'PctFam2Par', 'PctKids2Par', 'PctYoungKids2Par', 'PctTeen2Par', 'PctWorkMomYoungKids', 'PctWorkMom', 'NumKidsBornNeverMar', 'PctKidsBornNeverMar', 'NumImmig', 'PctImmigRecent', 'PctImmigRec5', 'PctImmigRec8', 'PctImmigRec10', 'PctRecentImmig', 'PctRecImmig5', 'PctRecImmig8', 'PctRecImmig10', 'PctSpeakEnglOnly', 'Pct

In [183]:
# Remove irrelevant features
del all_features[104:]
del all_features[0:2]
features_to_remove = ['racepctblack', 'racePctWhite', 'racePctAsian', 'racePctHisp', 'whitePerCap', 'blackPerCap', 'indianPerCap', 'AsianPerCap', 'OtherPerCap', 'HispPerCap', 'NumImmig', 'PctImmigRecent', 'PctImmigRec5', 'PctImmigRec8', 'PctImmigRec10', 'PctRecentImmig', 'PctRecImmig5', 'PctRecImmig8', 'PctRecImmig10']
all_features = [i for i in all_features if i not in features_to_remove]

print(all_features)
print('\nNumber of features after removing unnecessary features: ', len(all_features))

['population', 'householdsize', 'agePct12t21', 'agePct12t29', 'agePct16t24', 'agePct65up', 'numbUrban', 'pctUrban', 'medIncome', 'pctWWage', 'pctWFarmSelf', 'pctWInvInc', 'pctWSocSec', 'pctWPubAsst', 'pctWRetire', 'medFamInc', 'perCapInc', 'NumUnderPov', 'PctPopUnderPov', 'PctLess9thGrade', 'PctNotHSGrad', 'PctBSorMore', 'PctUnemployed', 'PctEmploy', 'PctEmplManu', 'PctEmplProfServ', 'PctOccupManu', 'PctOccupMgmtProf', 'MalePctDivorce', 'MalePctNevMarr', 'FemalePctDiv', 'TotalPctDiv', 'PersPerFam', 'PctFam2Par', 'PctKids2Par', 'PctYoungKids2Par', 'PctTeen2Par', 'PctWorkMomYoungKids', 'PctWorkMom', 'NumKidsBornNeverMar', 'PctKidsBornNeverMar', 'PctSpeakEnglOnly', 'PctNotSpeakEnglWell', 'PctLargHouseFam', 'PctLargHouseOccup', 'PersPerOccupHous', 'PersPerOwnOccHous', 'PersPerRentOccHous', 'PctPersOwnOccup', 'PctPersDenseHous', 'PctHousLess3BR', 'MedNumBR', 'HousVacant', 'PctHousOccup', 'PctHousOwnOcc', 'PctVacantBoarded', 'PctVacMore6Mos', 'MedYrHousBuilt', 'PctHousNoPhone', 'PctWOFullPlu

In [184]:
# Initial random forest model using all features
X = df[all_features]
y = df['total_crime']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

model = RandomForestRegressor(random_state=2)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=2)

In [185]:
# R^2 and mean absolute error score
y_pred = model.predict(X_test)

r_squared = metrics.r2_score(y_test, y_pred)
print('R-Squared Score:', r_squared)

mae = metrics.mean_absolute_error(y_test, y_pred)
print('Mean Absolute Error:', mae)

R-Squared Score: 0.9587636397597512
Mean Absolute Error: 912.3118940609953


In [186]:
# Check feature importance
feature_imp = pd.Series(model.feature_importances_, index=all_features).sort_values(ascending=False)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(feature_imp)

numbUrban                0.211628
NumUnderPov              0.182961
population               0.178533
NumKidsBornNeverMar      0.127977
NumStreet                0.112703
PctUsePubTrans           0.048384
NumInShelters            0.039377
HousVacant               0.035404
PctForeignBorn           0.009320
PctPersDenseHous         0.007341
OwnOccQrange             0.003648
PctHousOccup             0.002820
OwnOccHiQuart            0.002796
PctHousOwnOcc            0.002144
MalePctDivorce           0.002042
PopDens                  0.001697
PctBornSameState         0.001650
MedRent                  0.001553
RentQrange               0.001450
PctBSorMore              0.001320
PctEmplManu              0.001275
PctPersOwnOccup          0.001263
PctYoungKids2Par         0.001005
RentLowQ                 0.000986
PersPerRentOccHous       0.000956
householdsize            0.000954
PctVacantBoarded         0.000935
PctKids2Par              0.000929
FemalePctDiv             0.000910
MedRentPctHous

In [187]:
# Use the top 5 features to run our model again
list_of_features = feature_imp.index.tolist()
top_5_features = list_of_features[:5]
top_5_features

['numbUrban', 'NumUnderPov', 'population', 'NumKidsBornNeverMar', 'NumStreet']

In [188]:
# 2nd model using top 5 features based on feature importance
X = df[top_5_features]
y = df['total_crime']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

model = RandomForestRegressor(random_state=2)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=2)

In [189]:
# R^2 and mean absolute error score
y_pred = model.predict(X_test)

r_squared = metrics.r2_score(y_test, y_pred)
print('R-Squared Score:', r_squared)

mae = metrics.mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

R-Squared Score: 0.9341716477079319
Mean Absolute Error: 980.7956982343499


In [190]:
# Save model
# X = df[top_5_features]
# y = df['total_crime']

# model = RandomForestRegressor(random_state=2)
# model.fit(X, y)
# filename = "crime_model.pkl"
# pickle.dump(model, open(filename, 'wb'))

In [191]:
actual_vs_pred = X_test
actual_vs_pred['actual'] = y_test
actual_vs_pred['prediction'] = y_pred
actual_vs_pred['abs_error'] = abs(actual_vs_pred['prediction'] - actual_vs_pred['actual'])

actual_vs_pred.head()

,numbUrban,NumUnderPov,population,NumKidsBornNeverMar,NumStreet,actual,prediction,abs_error
802,50323,5173,50323,886,0,3197.0,2707.27,489.73
2137,0,592,11795,98,0,366.0,432.69,66.69
1112,0,2881,12291,194,0,757.0,676.10,80.90
2168,26243,901,26243,201,0,723.0,787.75,64.75
1812,3244,310,11622,56,0,63.0,302.56,239.56


In [192]:
actual_vs_pred['abs_error'].min()

2.0

In [193]:
fig = px.scatter(actual_vs_pred, x='actual', y='prediction', template='presentation')
fig.update_layout(xaxis_title="Actual Total Crime", yaxis_title="Predicted Total Crime", xaxis_range=[0,300000], yaxis_range=[0,300000])
fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1}])
fig.update_layout(
    title={
        'text': 'Compare Actual Total Crime to Prediction',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

In [194]:
fig = px.histogram(actual_vs_pred, x='abs_error', nbins=10, template='presentation')

fig.show()